# Visualization

- Designers like to go to design talks and share knowledge. There must be some nearby companies that also do design.
- 30% of the company staff have at least 1 child.
- Developers like to be near successful tech startups that have raised at least 1 Million dollars.
- Executives like Starbucks A LOT. Ensure there's a starbucks not too far.
- Account managers need to travel a lot.
- Everyone in the company is between 25 and 40, give them some place to go party.
- The CEO is vegan.
- If you want to make the maintenance guy happy, a basketball stadium must be around 10 Km.
- The office dog—"Dobby" needs a hairdresser every month. Ensure there's one not too far away.

In [1]:
from dotenv import load_dotenv
import os
import requests
import geopandas as gpd
import pandas as pd
from cartoframes.viz import Map, Layer, popup_element, basemaps
from cartoframes.auth import Credentials
from cartoframes.auth import set_default_credentials
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster

In [2]:
companies = pd.read_csv("../data/companies_cleaned.csv")
load_dotenv()
token_fsq = os.getenv("token_fsq")

In [3]:
companies

,Name,Number of employees,Description,Total money (in million $),Main address,City,Latitude,Longitude
0,ngmoco,150,Games for the iPhone,44.8,185 Berry Street,San Francisco,37.770510,-122.401581
1,Thumbplay,70,"Music, Videos, Games for Mobile Devices",41.5,599 Broadway,New York,40.725435,-73.997395
2,OMGPOP,50,Free online multiplayer game,16.6,SoHo,New York,40.723384,-74.001704
3,CyberArts,55,Gaming software developer,16.3,"51 Arbor St,",San Francisco,37.737609,-122.435399
4,Zynga,115,Network of gaming applications,8.0,365 Vermont St.,San Francisco,37.765158,-122.404234
5,Serious Business,22,Social Game Development Studio,4.0,580 Market Street,San Francisco,37.789321,-122.401362
6,Exent,100,Games on Demand Leader,3.0,632 W. 28th St.,New York,40.752380,-74.005568
7,Exent,100,Games on Demand Leader,3.0,685 Market Street,San Francisco,37.787646,-122.402759
8,GameLayers,5,Multiplayer Networked Games,2.0,1 Lombard Street,San Francisco,37.803847,-122.402468
9,Shattered Reality Interactive,10,"Online Social Gaming, MMO",1.1,490 2nd Street,San Francisco,37.783046,-122.394067


In [4]:
def call_FSQ(category):
    
    categories= {"vegan" : 13377,
                 "daycare" : 11026,
                 "night club" : 10032,
                 "airport" : 19031,
                 "bus" : 19042,
                 "metro" : 19046,
                 "train" : 19047,
                 "tram" : 19050,
                 "basket" : 18006
                }
    lst=[]
    
    for i in range(0,13):
              
        categ = categories[f"{category}"]
        latitude = companies.iloc[i].loc['Latitude']
        longitude = companies.iloc[i].loc['Longitude']

    
        url = f"https://api.foursquare.com/v3/places/search?ll={latitude}%2C{longitude}&categories={categ}&radius=500&limit=10"

        headers = {"accept": "application/json",
                  "Authorization" : token_fsq}

        response = requests.get(url, headers=headers).json()

        for i in response["results"]:
            name = i["name"]
            distance = i["distance"]
            address =  i["location"]["formatted_address"]
            lat = i["geocodes"]["main"]["latitude"]
            long = i["geocodes"]["main"]["longitude"]
            type_ = {"typepoint": 
                                {"type": "Point", 
                                "coordinates": [lat, long]}}

            lst.append({"name":name, "lat":lat, "lon":long, "distance":distance, "type":type_})

    category = pd.DataFrame(lst)
    category = gpd.GeoDataFrame(category, geometry=gpd.points_from_xy(category["lon"], category["lat"]))
    
    mapa = Map(Layer(category, "color:purple", popup_hover=[popup_element("name", "Restaurants near each location")]), basemap=basemaps.voyager)

    return mapa


In [30]:
def get_table(row, category):
    
    categories= {"vegan" : 13377,
                 "daycare" : 11026,
                 "night club" : 10032,
                 "airport" : 19031,
                 "bus" : 19042,
                 "metro" : 19046,
                 "train" : 19047,
                 "tram" : 19050,
                }

    latitude = companies.iloc[row].loc['Latitude']
    longitude = companies.iloc[row].loc['Longitude']
    categ = categories[category]

    url = f"https://api.foursquare.com/v3/places/search?ll={latitude}%2C{longitude}&categories={categ}&radius=500&limit=10"

    headers = {"accept": "application/json",
                          "Authorization" : token_fsq}

    response = requests.get(url, headers=headers).json()

    lst=[]
    
    for i in response["results"]:
        name = i["name"]
        distance = i["distance"]
        address =  i["location"]["formatted_address"]
        lat = i["geocodes"]["main"]["latitude"]
        long = i["geocodes"]["main"]["longitude"]
        type_ = {"typepoint": 
                            {"type": "Point", 
                            "coordinates": [lat, long]}}

        lst.append({"name":name, "lat":lat, "lon":long, "distance":distance, "type":type_})

    df = pd.DataFrame(lst)
    df.to_csv(f"../data/{row}{category}.csv", index=False)
        
    return

In [31]:
categories= {"vegan" : 13377,
             "daycare" : 11026,
             "night club" : 10032,
             "airport" : 19031,
             "bus" : 19042,
             "metro" : 19046,
             "train" : 19047,
             "tram" : 19050,
                }

In [38]:
for i in range(len(companies)):
    for j in categories.keys():
        get_table(i, j)